Computed Properties have values that are derived from existing item properties.
You can add a single index on these propertie or use them as part of a composite index for increased performance.
A Container can have a maximum of 20 computed properties.
It is highly recommended to use prefix or suffix to all computed property names.

In [11]:
#r "nuget: Newtonsoft.Json"
#r "nuget: Microsoft.Azure.Cosmos"
using Microsoft.Azure.Cosmos;
using System.Collections.ObjectModel;

Installed Packages Microsoft.Azure.Cosmos, 3.50.0 Newtonsoft.Json, 13.0.3

In [12]:
var endpoint = "https://localhost:8081";
var key ="C2y6yDjf5/R+ob0N8A7Cgv30VRDJIWEHLM+4QDU5DE2nQ9nDuVTqobD4b8mGGyPMbIZnqyMsEcaGQy67XIw/Jw==";
var client = new CosmosClient(endpoint, key);
var db = client.GetDatabase("StackOverflow");
var postContainer = db.GetContainer("Posts");

In [16]:
var props = await postContainer.ReadContainerAsync();
props.Resource.ComputedProperties = new System.Collections.ObjectModel.Collection<ComputedProperty>
{
    new ComputedProperty
    {
        Name = "cp_openindays",
        Query = "SELECT VALUE c.ClosedDate = null? 0: DateTimeDiff('d',c.CreatedOn, c.ClosedDate) FROM c"   
    }
};
props.Resource.IndexingPolicy.IncludedPaths.Add(new IncludedPath { Path = "/cp_openindays/?" });

var cont = await postContainer.ReplaceContainerAsync(props);

In [22]:
public class Post    
{
    public string id { get; set; }
    public int PostId { get; set; }
    public string PostBody { get; set; }
    public string Title { get; set; }
    public int ViewCount { get; set; }
    public int AnswerCount { get; set; }
    public int CommentCount { get; set; }
    public int FavoriteCount { get; set; }
    public int AcceptedAnswerId { get; set; }
    public DateTime? CreatedOn { get; set; }
    public DateTime? ClosedDate { get; set; }
    public int OwnerUserId { get; set; }
    public string OwnerDisplayName { get; set; }
    public string PostType { get; set; }
    public int Score { get; set; }
    public string Tags { get; set; }
}

Following query uses the system function

In [26]:
var cmd = "SELECT c.PostId FROM c WHERE DateTimeDiff('d',c.CreatedOn, c.ClosedDate) > 10";
var postQuery = new QueryDefinition(cmd);
var iterator = postContainer.GetItemQueryIterator<Post>(postQuery);
var results = new List<Post>();

while (iterator.HasMoreResults)
{
    var response = await iterator.ReadNextAsync();    
    results.AddRange(response);
    response.RequestCharge.Display(); // Display the request charge of the response
}

14.28

Following query uses the new computed property

In [25]:
var cmd = "SELECT c.PostId FROM c WHERE c.cp_openindays > 10";
var postQuery = new QueryDefinition(cmd);
var iterator = postContainer.GetItemQueryIterator<Post>(postQuery);
var results = new List<Post>();

while (iterator.HasMoreResults)
{
    var response = await iterator.ReadNextAsync();    
    results.AddRange(response);
    response.RequestCharge.Display(); // Display the request charge of the response
}

11.48